In [42]:
import argparse
import tensorflow as tf
import numpy as np
import cv2
import time
import matplotlib.pyplot as plt

from utils.image_reader import ImageReader
from tqdm import trange
from utils.config import Config
from model import ICNet, ICNet_BN

# Setup configurations

In [43]:
model_config = {'train': ICNet, 'trainval': ICNet, 'train_bn': ICNet_BN, 'trainval_bn': ICNet_BN, 'others': ICNet_BN}

# Choose dataset here, but remember to use `script/downlaod_weight.py` first
dataset = 'culane'
filter_scale = 1
    
class InferenceConfig(Config):
    def __init__(self, dataset, is_training, filter_scale):
        Config.__init__(self, dataset, is_training, filter_scale)
    
    # You can choose different model here, see "model_config" dictionary. If you choose "others", 
    # it means that you use self-trained model, you need to change "filter_scale" to 2.
    model_type = 'others'

    # Set pre-trained weights here (You can download weight from Google Drive) 
    model_weight = './snapshots/model.ckpt-60000'
    
    # Define default input size here
    INFER_SIZE = (1024, 2048, 3)
                  
cfg = InferenceConfig(dataset, is_training=False, filter_scale=filter_scale)
# cfg.display()


# model_config = {'train': ICNet, 'trainval': ICNet, 'train_bn': ICNet_BN, 'trainval_bn': ICNet_BN, 'others': ICNet_BN}
# dataset = 'culane'
# filter_scale = 1
# model = "others"
# model_weight = './snapshots/model.ckpt-60000'
# INFER_SIZE = (1024, 2048, 3)

# cfg = Config(dataset=dataset, is_training=False, filter_scale=filter_scale)


Setup configurations...


# Create graph, session, and restore weights

In [44]:
# Create graph here 
model = model_config[cfg.model_type]
net = model(cfg=cfg, mode='inference')

# Create session & restore weights!
net.create_session()
net.restore(cfg.model_weight)


# model = model_config[model]
# reader = ImageReader(cfg=cfg, mode='inference')
# net = model(cfg=cfg, mode='inference')

# net.create_session()
# net.restore(model_weight)

ValueError: Variable conv1_1_3x3_s2/weights already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "/root/atlast_root/home/vsalova/lane_lines/ICNet-tensorflow-master/network.py", line 142, in make_var
    return tf.get_variable(name, shape, trainable=self.trainable)
  File "/root/atlast_root/home/vsalova/lane_lines/ICNet-tensorflow-master/network.py", line 177, in conv
    kernel = self.make_var('weights', shape=[k_h, k_w, c_i, c_o])
  File "/root/atlast_root/home/vsalova/lane_lines/ICNet-tensorflow-master/network.py", line 26, in layer_decorated
    layer_output = op(self, layer_input, *args, **kwargs)


# Run segmentation on single image

In [ ]:
im1 = cv2.imread('./data/input/cityscapes1.png')

if im1.shape != cfg.INFER_SIZE:
    im1 = cv2.resize(im1, (cfg.INFER_SIZE[1], cfg.INFER_SIZE[0]))

results1 = net.predict(im1)
overlap_results1 = 0.5 * im1 + 0.5 * results1[0]
vis_im1 = np.concatenate([im1/255.0, results1[0]/255.0, overlap_results1/255.0], axis=1)

plt.figure(figsize=(20, 15))
plt.imshow(vis_im1)

In [ ]:
im2 = cv2.imread('./data/input/cityscapes2.png')
results2 = net.predict(im2)

im2 = cv2.cvtColor(im2, cv2.COLOR_BGR2RGB)
overlap_results2 = 0.5 * im2 + 0.5 * results2[0]

vis_im2 = np.concatenate([im2/255.0, results2[0]/255.0, overlap_results2/255.0], axis=1)

plt.figure(figsize=(20, 15))
plt.imshow(vis_im2)

# Test inference speed

In [ ]:
elapsed_times = []

for i in range(50):
    start_t = time.time()
    
    _ = net.predict(im1)
    
    duration = time.time() - start_t
    
    if i > 0:
        elapsed_times.append(duration)

print('Average time: {:.4f}, about {:.6f} fps'.format(np.mean(elapsed_times), 1/np.mean(elapsed_times)))